Import relevant packages:

In [ ]:
import spacy
import re

Define a function to remove special unicode characters and other non-relevant N-grams from each article:

In [1]:
def remove_clutter(text):
    #Trying to remove special unicode characters
    text = re.sub(r'\\x[A-Za-z0-9_]{2}', '', text)
    
    #Trying to remove video annotation
    text = re.sub(r'VIDEO([0-9]|[0-9]{2}):[0-9]{4}:[0-9]{2}', ' ', text)
    
    # Remove all "â" with space:
    text = re.sub(r'â',' ', text)
    text = re.sub(r'Â',' ', text)
    
    # Remove all "Getty images":
    text = re.sub(r'getty imgages','', text)
    return text

Load the English language model instance in spaCy:

In [ ]:
nlp = spacy.load('en_core_web_lg')


Define a function to clean all our articles in a specific column in a dataset and return the tokenized data


In [ ]:
def cleaning(df,column):
    
    # Create an empty list to store tokens:
    tokens = []

    #df[column].map(lambda article: remove_clutter(article))

    ## Apply the remove_clutter function on all articles in the df:
    df[column].apply(remove_clutter())
    
    # Iterate over all articles in the dataframe and create a nlp object for each:
    for article in nlp.pipe(df[column], disable=["tagger", "parser"]):
        
        # Store all cleaned tokens in a list using list comprehension:
        article_tok = [token.lemma_.lower() for token in article if not token.is_stop and token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV', 'VERB'] and token.ent_type_ not in ["PERSON", "MONEY", "PERCENT", "LOC", "DATE", "TIME", "QUANTITY", "ORDINAL"]]:
    
        # Insert list of tokens into the list "tokens":
        tokens.append(article_tok)
        
    # Insert a column in the dataframe with all tokens for each article:
    df['tokens'] = tokens
    
    # Insert tokens as a string to the dataframe:
    df["clean_articles"] = data["tokens"].map(lambda row: " ".join(row))

    # Return the df
    return df

Call the cleaning function of our dataset:

In [2]:
cleaning(df,'articles')


NameError: ignored

Example: How to entity recognition work:

In [ ]:
doc = nlp("SOME TEXT")

In [ ]:
sentence_spans = list(doc.sents)
displacy.serve(sentence_spans, style="dep")

displacy.serve(doc, style="dep")